In [5]:
import os


In [6]:
import argparse
import os 

def revcomplE(word):
	revComp=''
	transdict={'A':'T','C':'G','G':'C','T':'A'}
	index=0
	while index <len(word):
		if word[index] == 'E' :#if is methylated C
			if index <len(word)-1 and word[index+1]=='G': #if G is next to it
				revComp += 'GE'
				index+=2   
			else:
				revComp += 'G'
				index+=1
		else:
			try: 
				revComp += transdict[word[index]]
			except KeyError:
				revComp += 'N'
			index+=1		 
	return revComp[::-1]

def revcomplEF(word):
	revComp=''
	transdict={'A':'T','C':'G','G':'C','T':'A','E':'F','F':'E'}
	index=0
	while index <len(word):
		try:
			revComp += transdict[word[index]]
		except KeyError:
			revComp += 'N'
		index += 1		 
	return revComp[::-1]

def main():
	parser = argparse.ArgumentParser()
	parser.add_argument("-gd", "--genomedir", help="a directory that contains genome fasta sequence files, ideally one chromosome per file")
	parser.add_argument("-k", "--klength",type =int, help="integer, the length of k-mers")
	parser.add_argument('--typeEF', dest='typeEF',action='store_true', help="using typeEF, default is typeE")
	parser.set_defaults(typeEF=False)

	args = parser.parse_args()
	

	genomedir = args.genomedir
	if genomedir == None:
		print "GENOMEDIR is missing, please specify"
		return
	try:
		k = int(args.klength)
	except:
		print "KLENGTH must be specified"
		return

	typeEF = args.typeEF
	print "GENOMEDIR is",genomedir
	print "klength is", k
	print "Using typeEF",typeEF

	kmercounts = {} # 
	total = 0
	for file in os.listdir(genomedir):
		print "Reading ...",file
		for line in open(genomedir+'/'+file):
			if line[0] == ">": # this is the name, skip
				continue
			seq = line.strip().upper()
			print len(seq)
			
			#count the kmers
			kmers = [seq[i:i+k] for i in range(len(seq)-k)]
			for i in range(len(kmers)):
				kmer = kmers[i]
				if "N" in kmer:
					continue
				try: 
					kmercounts[kmer] += 1
				except KeyError:
					kmercounts[kmer] = 1
			
			if typeEF: #check if type E or EF
				seq = revcomplEF(seq)
			else:
				seq = revcomplE(seq)

			kmers = [seq[i:i+k] for i in range(len(seq)-k)]
			for i in range(len(kmers)):
				kmer = kmers[i]
				if "N" in kmer:
					continue
				try: 
					kmercounts[kmer] += 1
				except KeyError:
					kmercounts[kmer] = 1

	#output
	outfile = ""
	if typeEF:
		outfile = "background_typeEF-"+str(k) +".tsv"
	else:
		outfile = "background_typeE-"+str(k) +".tsv"
	target = open(outfile,'w')
	target.write("TOTAL\t"+str(total)+'\n')
	for kmer in kmercounts:
		target.write(kmer+'\t'+str(kmercounts[kmer])+'\n')
	target.close()
		


	return 


if __name__ == "__main__":
	main()

In [11]:
genomedir = "./samplegenome/"
k = 8
typeEF = True

kmercounts = {} # 
total = 0
for file in os.listdir(genomedir):
    print "Reading ...",file
    for line in open(genomedir+'/'+file):
        if line[0] == ">": # this is the name, skip
            continue
        seq = line.strip()
        
        #count the kmers
        for i in range(len(seq) - k):
            kmer = seq[i:i+k]
            if "N" in kmer:
                continue
            try: 
                kmercounts[kmer] += 1
            except KeyError:
                kmercounts[kmer] = 1
            total += 1
        
        
        if typeEF: #check if type E or EF
            seq = revcomplEF(seq)
        else:
            seq = revcomplE(seq)
        
        for i in range(len(seq) - k):
            kmer = seq[i:i+k]
            if "N" in kmer:
                continue
            try: 
                kmercounts[kmer] += 1
            except KeyError:
                kmercounts[kmer] = 1
            total += 1
           
        
#output
outfile = ""
if typeEF:
    outfile = "background_typeEF-"+str(k) +".tsv"
else:
    outfile = "background_typeE-"+str(k) +".tsv"
target = open(outfile,'w')
target.write("TOTAL\t"+str(total)+'\n')
for kmer in kmercounts:
    target.write(kmer+'\t'+str(kmercounts[kmer])+'\n')
target.close()
    

Reading ... chr2
Reading ... chr1


In [41]:
seq = "A"*2000000


In [42]:
import timeit

def substring(s,k):
    s = list(s)
    kmers = []
    for i in range(len(s)-k):
        kmer = s[i:i+k]
        kmers += [''.join(kmer)]
    return kmers

start_time = timeit.default_timer()
k = 8
kmers = substring(seq,8)


# code you want to evaluate
elapsed = timeit.default_timer() - start_time

print elapsed

2.58216118813


In [43]:

k = 8
kmers = []
for i in range(len(seq)-k):
    kmer = seq[i:i+k]
    kmers += [kmer]

# code you want to evaluate
elapsed = timeit.default_timer() - start_time

print elapsed

13.8980710506


In [44]:
start_time = timeit.default_timer()
kmers = [ seq[i:i+k] for i in range(len(seq)-k)]
print timeit.default_timer() - start_time


0.888291120529


In [66]:
typeEF = False

genomedir = "samplegenome/"
k = 8

seq = open("samplegenome/chr1").read().split('\n')[1]
print seq[:100]
print len(seq)



s_total_time = timeit.default_timer()

#count the kmers
start_time = timeit.default_timer()

kmers = [seq[i:i+k] for i in range(len(seq)-k)]
for i in range(len(kmers)):
    kmer = kmers[i]
    try: 
        kmercounts[kmer] += 1
    except KeyError:
        kmercounts[kmer] = 1
print "TIME 1",timeit.default_timer() - start_time


if typeEF: #check if type E or EF
    seq = revcomplEF(seq)
else:
    seq = revcomplE(seq)


start_time = timeit.default_timer()
for i in range(len(seq)-k):
    kmer = seq[i:i+k]
    try: 
        kmercounts[kmer] += 1
    except KeyError:
        kmercounts[kmer] = 1
print "TIME 2",timeit.default_timer() - start_time

total = 0
for kmer in kmercounts:
    if "N" in kmer:
        continue
    total += kmercounts[kmer]
    
#output
print "TOTAL time",timeit.default_timer() - s_total_time




ATGGGCGAACGACGGGAATTGAACCCGCGATGGTGAATTCACAATCCACTGCCTTAATCCACTTGGCTACATCCGCCCCTACGCTACTATCTATTCTTTT
154478
TIME 1 0.21578001976
TIME 2 0.126627206802
TOTAL time 0.499745130539


In [67]:
typeEF = False

genomedir = "samplegenome/"
k = 8

seq = open("samplegenome/chr1").read().split('\n')[1]
print seq[:100]
print len(seq)


s_total_time = timeit.default_timer()

#count the kmers


start_time = timeit.default_timer()

kmers = [seq[i:i+k] for i in range(len(seq)-k)]
for i in range(len(kmers)):
    kmer = kmers[i]
    try: 
        kmercounts[kmer] += 1
    except KeyError:
        kmercounts[kmer] = 1
print "TIME 1",timeit.default_timer() - start_time


if typeEF: #check if type E or EF
    seq = revcomplEF(seq)
else:
    seq = revcomplE(seq)


start_time = timeit.default_timer()
for i in range(len(seq)-k):
    kmer = seq[i:i+k]
    try: 
        kmercounts[kmer] += 1
    except KeyError:
        kmercounts[kmer] = 1
print "TIME 2",timeit.default_timer() - start_time

total = 0
for kmer in kmercounts:
    if "N" in kmer:
        continue
    total += kmercounts[kmer]
    
#output

print "TOTAL time",timeit.default_timer() - s_total_time



ATGGGCGAACGACGGGAATTGAACCCGCGATGGTGAATTCACAATCCACTGCCTTAATCCACTTGGCTACATCCGCCCCTACGCTACTATCTATTCTTTT
154478
TIME 1 0.199972867966
TIME 2 0.13326883316
TOTAL time 0.493828058243


In [68]:
from collections import Counter

In [71]:
seq = "ATGGGCGAACGACGGGAATTGAACCCGCGATGGTGAATTCACAATCCACTGCCTTAATCCACTTGGCTACATCCGCCCCTACGCTACTATCTATTCTTTT"
c = Counter()
for i in range(len(seq)-k):
    kmer = seq[i:i+k]
    c[kmer] += 1

In [72]:
c


Counter({'AACCCGCG': 1,
         'AACGACGG': 1,
         'AATCCACT': 2,
         'AATTCACA': 1,
         'AATTGAAC': 1,
         'ACAATCCA': 1,
         'ACATCCGC': 1,
         'ACCCGCGA': 1,
         'ACGACGGG': 1,
         'ACGCTACT': 1,
         'ACGGGAAT': 1,
         'ACTATCTA': 1,
         'ACTGCCTT': 1,
         'ACTTGGCT': 1,
         'ATCCACTG': 1,
         'ATCCACTT': 1,
         'ATCCGCCC': 1,
         'ATCTATTC': 1,
         'ATGGGCGA': 1,
         'ATGGTGAA': 1,
         'ATTCACAA': 1,
         'ATTGAACC': 1,
         'CAATCCAC': 1,
         'CACAATCC': 1,
         'CACTGCCT': 1,
         'CACTTGGC': 1,
         'CATCCGCC': 1,
         'CCACTGCC': 1,
         'CCACTTGG': 1,
         'CCCCTACG': 1,
         'CCCGCGAT': 1,
         'CCCTACGC': 1,
         'CCGCCCCT': 1,
         'CCGCGATG': 1,
         'CCTACGCT': 1,
         'CCTTAATC': 1,
         'CGAACGAC': 1,
         'CGACGGGA': 1,
         'CGATGGTG': 1,
         'CGCCCCTA': 1,
         'CGCGATGG': 1,
         'CGCTAC